In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, concatenate

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('dataset/yearRecord.csv')

In [4]:
df.head()

,AssetTypeID,AssetType,YearsAfterInstallation,WorkDescription,Spend_ExGST
0,89,DUCTED SPLIT SYSTEM,25.0,Ex Taste of Korea AC reactive,0.0
1,10,AIR FAN - EXHAUST,4.0,General exhaust fan isolator i,0.0
2,10,AIR FAN - EXHAUST,4.0,General exhaust fan isolator i,0.0
3,89,DUCTED SPLIT SYSTEM,20.0,LG- Heat Pump NO# 2 Repairs,0.0
4,89,DUCTED SPLIT SYSTEM,19.0,L17 Quorom Comms. Room PCB Rep,0.0


In [5]:
X = df[['AssetType','YearsAfterInstallation']]
Y = df[['WorkDescription','Spend_ExGST']]

In [6]:
num_unique_asset_types = X['AssetType'].nunique()

In [7]:
num_unique_years = X['YearsAfterInstallation'].nunique()

In [9]:
# Input layer for categorical variables
asset_type_input = Input(shape=(1,), name='asset_type_input')
year_input = Input(shape=(1,), name='year_input')

In [14]:
embedding_dim = 10  # You can adjust this based on your preference
num_unique_work_descriptions = Y['WorkDescription'].nunique()

In [18]:
asset_type_branches = []
for asset_type in X['AssetType'].unique():
    # Filtering data for the specific Asset Type
    filtered_data = X[X['AssetType'] == asset_type]

    # Creating separate branches
    asset_type_embedding = Embedding(input_dim=num_unique_asset_types, output_dim=embedding_dim)(asset_type_input)
    year_embedding = Embedding(input_dim=num_unique_years, output_dim=embedding_dim)(year_input)
    asset_type_flatten = Flatten()(asset_type_embedding)
    year_flatten = Flatten()(year_embedding)

    # Concatenate with other features
    concatenated_inputs = concatenate([asset_type_flatten, year_flatten])

    # Shared layers
    shared_dense = Dense(128, activation='relu')(concatenated_inputs)

    # WorkDescription prediction branch
    cleaned_asset_type = asset_type.replace(' ', '_')  # Replace spaces with underscores
    work_description_output = Dense(num_unique_work_descriptions, activation='softmax', name=f'{cleaned_asset_type}')(work_description_branch)
    # Spend_ExGST prediction branch
    spend_branch = Dense(128, activation='relu')(shared_dense)
    spend_output = Dense(1, activation='linear', name=f'{asset_type}')(spend_branch)

    # Create the model for each Asset Type
    model = Model(inputs=[asset_type_input, year_input], outputs=[work_description_output, spend_output])
    asset_type_branches.append(model)

ValueError: 'DUCTED SPLIT SYSTEM/' is not a valid root scope name. A root scope name has to match the following pattern: ^[A-Za-z0-9.][A-Za-z0-9_.\\/>-]*$